In [17]:
# Carga de librerias
import os
import pandas as pd
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv


In [18]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
# Carga de la documentación
documentation = pd.read_pickle("../APP/Documents/documentation.pickle")
documentation.head(3)

,page_content,metadata,embeddings
0,Visualizing\nVergadia816324_ffirs.indd 1 25-...,"{'source': '../APP/Documents/GCP.pdf', 'page': 2}","[0.017832092940807343, 0.02583819441497326, -0..."
1,Vergadia816324_ffirs.indd 2 25-02-2022 15:...,"{'source': '../APP/Documents/GCP.pdf', 'page': 3}","[-0.03665415197610855, 0.039491377770900726, -..."
2,Priyanka Vergadia\nIllustrated References for ...,"{'source': '../APP/Documents/GCP.pdf', 'page': 4}","[-0.01771504618227482, 0.07079385221004486, -0..."


In [8]:
# Pregunta de usuario
def question_user(q:str, embeddings=embeddings):
  data_question = pd.DataFrame()
  emb = []
  q_list = []
  emb.append(embeddings.embed_query(q))
  q_list.append(q)
  data_question["pregunta"] = q_list
  data_question["embedding_pregunta"] = emb
  return data_question

In [28]:
p = "Que es Cloud Storage?"
data_p = question_user(p)
data_p

,pregunta,embedding_pregunta
0,Que es Cloud Storage?,"[-0.00723279919475317, 0.024123195558786392, -..."


In [9]:
def distance_vector(x, y):
    return np.dot(np.array(x), np.array(y))

In [29]:
documentation["distance"] = documentation["embeddings"].apply(lambda x: distance_vector(data_p.iloc[0,-1],x))

In [31]:
documentation.head(3)

,page_content,metadata,embeddings,distance
678,• BigQuery is a serverless data warehouse that...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[0.06403540074825287, 0.06960034370422363, -0....",0.337325
554,of data analysts. It allows data analysts or d...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[-0.007997865788638592, 0.10090051591396332, -...",0.308750
562,of data analysts. It allows data analysts or d...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[-0.007997865788638592, 0.10090051591396332, -...",0.308750


In [32]:
documentation.sort_values(by="distance",ascending=False,inplace=True)

In [33]:
documentation.head(3)

,page_content,metadata,embeddings,distance
227,Storage 31\nFundamental characteristics of an ...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[-0.0017023786203935742, 0.010072107426822186,...",0.607445
230,You can use cloud storage to store and serve s...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[-0.05700237303972244, -0.009095263667404652, ...",0.595455
229,continuity requirements.• Data lakes — You c...,"{'source': '../APP/Documents/GCP.pdf', 'page':...","[-0.013028450310230255, 0.012346569448709488, ...",0.586917


In [34]:
response = documentation.head(10)

In [24]:
template = """
Eres Laia, una asistente virtual que recibe la siguiente pregunta: {question}
y tienes la siguiente información para responder: {context}.
Limitate a responder solamente con la información que recibes.
Responde de forma muy completa.
"""
prompt = PromptTemplate(
        input_variables=["question","context"],template=template
)

In [25]:
model = ChatOpenAI(temperature=0.0,
                   model_name = 'gpt-3.5-turbo-16k',
                   api_key=api_key)

In [26]:
llm = LLMChain(llm = model,prompt=prompt)

In [35]:
llm.run(question=p,context=response.iloc[:,1])

'El Cloud Storage es un servicio de almacenamiento en la nube que permite a los usuarios almacenar y acceder a sus datos de forma remota a través de Internet. Este servicio proporciona una forma segura y escalable de almacenar y respaldar datos, eliminando la necesidad de tener servidores físicos en el lugar.\n\nEl Cloud Storage utiliza la infraestructura de la nube para almacenar los datos de los usuarios en múltiples ubicaciones geográficas, lo que garantiza la disponibilidad y durabilidad de los datos. Los usuarios pueden acceder a sus datos desde cualquier dispositivo con conexión a Internet, lo que facilita el acceso y la colaboración en tiempo real.\n\nAdemás, el Cloud Storage ofrece características avanzadas como la capacidad de almacenar y recuperar grandes volúmenes de datos, la posibilidad de compartir archivos y carpetas con otros usuarios, y la opción de configurar políticas de retención y eliminación automática de datos.\n\nEn resumen, el Cloud Storage es un servicio de al